Genetic Algorithm - An algorithm that pits different solutions against each other and uses selection to "evolve" them into the best solution.

Genetic algorithms efficiently explore the solution space to discover optimal or near-optimal solutions.

By mimicking the process of natural selection, GAs can evolve solutions over time, improving their performance iteratively. 

---

**Key Terminology**

**Chromosomes** - potential solutions to a problem, typically coded as strings or arrays. Each element in a chromosome corresponds to a gene that determines a specific trait of the solution. 

**Crossover** - involves exchanging segments of the *'parent'* chromosomes to create new *'individuals'*. This operation introduces variability and allows the *'offspring'* to inherit beneficial traits from both *'parents'*.

**Mutation** - randomly altering one or more genes. 